# V3 Training & Evaluation

In [ ]:
!pip install datasets
!pip install transformers

import torch
import random
import torch.nn.functional as F
from torch.utils.data import DataLoader

from datasets import load_metric,load_dataset,Dataset

import transformers
from transformers import RobertaTokenizer, DataCollatorWithPadding,RobertaForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer, set_seed

import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split,StratifiedKFold
from tqdm.auto import tqdm, trange

import csv      
import gc
import re

import logging
import warnings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/si

In [ ]:
def compute_metrics(model, testing_dataloader):
    """computes F1 score and accuracy over dataset

    Args:
        model (any type): model for evaluation
        testing_dataloader (huggingface dataset): self explained

    Returns:
        f1 score
    """
    metric_f1 = load_metric("f1")

    model.eval()
    for batch in testing_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)  
        metric_f1.add_batch(predictions=predictions, references=batch["labels"])
        
    return metric_f1.compute(average='macro')['f1']

In [ ]:
BATCH_SIZE = 64
N_FOLDS = 5
N_EPOCHS = 3
WEIGHT_DECAY=0.1
LR=5e-5
MODEL_CHECKPOINT = 'roberta-base'
RANDOM_SEED = 42

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
transformers.logging.set_verbosity(transformers.logging.ERROR)

logging.disable(logging.ERROR)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning) 

# Helper function for reproducible behavior to set the seed in random, numpy, torch
set_seed(RANDOM_SEED)

## Data preprocessing

In [ ]:
data_babe = load_dataset("mediabiasgroup/BABE")['train'].to_pandas()
# Filter out sentences with no agreement
data_babe=data_babe[data_babe.label_bias!='No agreement']
# Rename and make label numeric
data_babe['label']=data_babe['label_bias'].apply(lambda x: 0 if x=='Non-biased' else 1)
data_babe.info()
data_babe.head()

data_babe[:10]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset babe downloaded and prepared to /root/.cache/huggingface/datasets/mediabiasgroup___babe/default/0.0.0/a62c13014c0a0666b78ffcf5a331e8044855fb362823df4ca729437b084b7d69. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3673 entries, 0 to 3673
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           3673 non-null   object
 1   news_link      3673 non-null   object
 2   outlet         3673 non-null   object
 3   topic          3673 non-null   object
 4   type           3673 non-null   object
 5   label_bias     3673 non-null   object
 6   label_opinion  3673 non-null   object
 7   biased_words   3673 non-null   object
 8   label          3673 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 287.0+ KB


,text,news_link,outlet,topic,type,label_bias,label_opinion,biased_words,label
0,"""Orange Is the New Black"" star Yael Stone is r...",https://www.foxnews.com/entertainment/australi...,Fox News,environment,right,Non-biased,Entirely factual,[],0
1,"""We have one beautiful law,"" Trump recently sa...",https://www.alternet.org/2020/06/law-and-order...,Alternet,gun control,left,Biased,Somewhat factual but also opinionated,"['bizarre', 'characteristically']",1
2,"...immigrants as criminals and eugenics, all o...",https://www.nbcnews.com/news/latino/after-step...,MSNBC,white-nationalism,left,Biased,Expresses writer’s opinion,"['criminals', 'fringe', 'extreme']",1
3,...we sounded the alarm in the early months of...,https://www.alternet.org/2019/07/fox-news-has-...,Alternet,white-nationalism,left,Biased,Somewhat factual but also opinionated,[],1
4,[Black Lives Matter] is essentially a non-fals...,http://feedproxy.google.com/~r/breitbart/~3/-v...,Breitbart,marriage-equality,nan,Biased,Expresses writer’s opinion,['cult'],1
5,[Democrats employ] their full arsenal to deleg...,https://thefederalist.com/2020/09/22/democrats...,Federalist,vaccine,nan,Biased,Expresses writer’s opinion,"['apparent', 'crime']",1
6,[Newsoms's] obsession with masks has created a...,https://thefederalist.com/2020/12/09/gavin-new...,Federalist,vaccine,nan,Biased,Expresses writer’s opinion,['obsession'],1
7,[Newsoms's] onslaught of propaganda ignores co...,https://thefederalist.com/2020/12/09/gavin-new...,Federalist,vaccine,nan,Biased,Expresses writer’s opinion,"['propaganda', 'vilifying', 'unimpeded']",1
8,[The police] now prefer to think of themselves...,http://feedproxy.google.com/~r/breitbart/~3/2R...,Breitbart,marriage-equality,nan,Biased,Expresses writer’s opinion,['wounds'],1
9,‘A new low’: Washington Post media critic blow...,https://www.alternet.org/2019/08/a-new-low-was...,Alternet,white-nationalism,left,Biased,Expresses writer’s opinion,"['blows', 'up', 'absurd', 'lies', 'nationalism...",1


In [ ]:
data_v3 = pd.read_csv('/content/data_v3.csv')
data_v3.info()
data_v3.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      310 non-null    object
 1   label   310 non-null    int64 
 2   text    310 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.4+ KB


,id,label,text
0,cdc102f5-88b2-4f1c-ba7d-9970ac9669cd,0,"""You think I'm joking,"" he continued. """
1,cb534c0b-5bea-4ade-b5da-cd766f6d5ab7,0,"Speaking about the ""Divine Nine"" — the nine bl..."
2,1c03f90d-54c0-4a9b-9309-1818488d294f,0,"I know where the power is."""
3,be0165c0-fd53-438a-8fda-bb04df7d12e2,1,President Joe Biden made a bizarre race-relate...
4,cd0a2226-c30d-428f-b29d-1e6344b618ac,0,"""And, by the way, you know I'm not — I may be ..."


In [ ]:
data_full = pd.concat([data_v3, data_babe], join = 'inner').reindex()
data_full.info()
data_full.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3983 entries, 0 to 3673
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   3983 non-null   int64 
 1   text    3983 non-null   object
dtypes: int64(1), object(1)
memory usage: 93.4+ KB


,label,text
0,0,"""You think I'm joking,"" he continued. """
1,0,"Speaking about the ""Divine Nine"" — the nine bl..."
2,0,"I know where the power is."""
3,1,President Joe Biden made a bizarre race-relate...
4,0,"""And, by the way, you know I'm not — I may be ..."


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_CHECKPOINT)

## Training

In [ ]:
def train_and_evaluate(data, run_name):
  skfold = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)

  tok = tokenizer(list(data['text']),truncation=True, padding=True, max_length=128)

  data = pd.DataFrame({'input_ids':tok['input_ids'],'attention_mask':tok['attention_mask'],'label':data['label']})
  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  training_args = TrainingArguments(
    output_dir = './',
    num_train_epochs=N_EPOCHS,
    per_device_train_batch_size=32,  
    logging_steps=20,
    disable_tqdm = False,
    save_total_limit=2,
    weight_decay=WEIGHT_DECAY,
    run_name=run_name,
    learning_rate=LR)
  
  scores = []

  for train_index, val_index in skfold.split(data['input_ids'],data['label']):
    token_train = Dataset.from_dict(data.iloc[train_index])
    token_valid = Dataset.from_dict(data.iloc[val_index])
    
    model=RobertaForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,
                      tokenizer=tokenizer)
    trainer.train()

    #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=32, collate_fn=data_collator)
    scores.append(compute_metrics(model, eval_dataloader))
    print(scores[-1])
  
  print(f"F1 score for {run_name}: {np.mean(scores)} +/- {np.std(scores)}")
  return scores

#### V3 only

In [ ]:
train_and_evaluate(data_v3, "V3 only")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
20,0.590500


<ipython-input-2-c2be2f0b1c7b>:11: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_f1 = load_metric("f1")


0.41509433962264153


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

0.7074410163339383


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
20,0.541300


0.40952380952380957
F1 score for V3 only: 0.4724495689451345 +/- 0.11751552964314349


[0.41509433962264153,
 0.41509433962264153,
 0.41509433962264153,
 0.7074410163339383,
 0.40952380952380957]

#### BABE only

In [ ]:
train_and_evaluate(data_babe, "BABE Only")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
20,0.664200
40,0.541500
60,0.495200
80,0.514600
100,0.468200
120,0.380700
140,0.347600
160,0.293600
180,0.354300
200,0.230500


0.8102648102648102


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
20,0.629300
40,0.539600
60,0.528500
80,0.517000
100,0.547600
120,0.393100
140,0.361300
160,0.356300
180,0.350900
200,0.280800


...

0.8037214484679666


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
20,0.674200
40,0.549500
60,0.514100
80,0.444200
100,0.422500
120,0.315600
140,0.326200
160,0.369500
180,0.361600
200,0.262900


0.8157037720373391
F1 score for BABE Only: 0.8194833338586787 +/- 0.013352449887641716


[0.8102648102648102,
 0.8419307951292585,
 0.8257958433940183,
 0.8037214484679666,
 0.8157037720373391]

#### BABE + V3 combined

In [ ]:
train_and_evaluate(data_full, "BABE+V3")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
20,0.695400
40,0.652100
60,0.540600
80,0.490200
100,0.461700
120,0.395500
140,0.387200
160,0.364600
180,0.334600
200,0.337200


0.8031874477623282


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
20,0.629900
40,0.554400
60,0.552300
80,0.479300
100,0.458800
120,0.329200
140,0.352200
160,0.357800
180,0.350600
200,0.378700


...

0.8435534591194969


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
20,0.651600
40,0.567100
60,0.534700
80,0.483100
100,0.417600
120,0.334600
140,0.337800
160,0.346100
180,0.328400
200,0.335000


0.805140758873929
F1 score for BABE+V3: 0.8244664742512237 +/- 0.01715171317655662


[0.8031874477623282,
 0.8302854055428426,
 0.8401652999575222,
 0.8435534591194969,
 0.805140758873929]